In [4]:
import pandas as pd
import geopandas as gpd
import fiona

In [3]:
df = pd.read_csv('/Users/nicholashorton/Documents/GIS Research Project/Advanced Search 7-31-2025.csv')

In [14]:
# List all layers in the geodatabase
gdb_path = "Final_Project/Final_Project.gdb"
layers = fiona.listlayers(gdb_path)
print(layers)

# # Read a specific layer
gdf = gpd.read_file(gdb_path, layer="HouseFinal")

['HouseFinal_Withou_Statistics', 'OLS_Coefficient', 'OLS_Diagnostic', 'HouseFinal_SpatialOutlierDetection', 'HouseFinal_SpatialOutlierDetection1', 'tl_2019_12_tract_Project', 'Census_Tracts', 'Census_Tracts_XYTableToPoint', 'HouseFinal_SpatialOutlierDetection2', 'HouseFinal', 'HouseFinal_OrdinaryLeastSquares', 'HouseFinal_GWR', 'HouseFinal_HotSpots', 'test', 'test_label', 'Random_Sample', 'Random_Sample_GWR', 'Random_Sample_GWR2', 'Random_Sample_MGWR', 'Random_Sample_OrdinaryLeastSquares']


# Clean up df

In [17]:
df.columns

Index(['PARCEL_NUMBER', 'TOTAL_LIVING_SQFT', 'TOTAL_GROSS_SQFT', 'CENSUS',
       'YEAR_BUILT_BLD1', 'TOTAL_LIVING_UNITS', 'ACREAGE', 'CNTY_JST_VALUE',
       'CNTY_ASD_VALUE', 'CNTY_TAXABLE_VALUE', 'JUST_LAND', 'JUST_BUILDING',
       'JUST_EXTRA_FEATURES', 'PROPERTY_USE', 'SALES_DATE', 'PRICE', 'SEAWALL',
       'FRONTAGE', 'VIEWS', 'SUBSIDENCE_YN', 'DLHL_YN', 'FLOOR_NUM',
       'LAND_USE', 'EVAC_ZONE', 'TOTAL_BUILDINGS', 'POOL', 'WATERFRONT_YN',
       'XFEAT_DESCRIPTION', 'HEATED_AREA_SQFT', 'GROSS_AREA_SQFT', 'QUALITY',
       'DPR_BLD_VAL', 'BALCONY_PORCH', 'CLUBHOUSE', 'COMPLEX_VIEW',
       'EFFECTIVE_AGE', 'COOLING', 'UNIT_VIEW', 'EXTERIOR_WALLS', 'HEATING',
       'LIVING_UNITS', 'MARINA', 'PARKING_GARAGE', 'ROOF_COVER', 'TENNIS',
       'LOCATION'],
      dtype='object')

# Clean up gdf

In [ ]:
columns_to_drop = [
    'CENSUS','JUST_LAND', 'JUST_BUILDING',
       'JUST_EXTRA_FEATURES','EVAC_ZONE'
]

# Drop unwanted columns
df = df.drop(columns=columns_to_drop, errors='ignore')

# Columns to fill missing values and create dummies
cat_cols = [
    'FRONTAGE', 'VIEWS', 'SUBSIDENCE_YN', 'DLHL_YN', 'EVAC_ZONE', 'POOL', 'WATERFRONT_YN', 'QUALITY', 'BALCONY_PORCH',
    'CLUBHOUSE', 'COMPLEX_VIEW', 'COOLING', 'UNIT_VIEW', 'EXTERIOR_WALLS', 'HEATING', 'LIVING_UNITS', 'MARINA',
    'PARKING_GARAGE', 'ROOF_COVER', 'TENNIS', 'SEAWALL'
]

# Fill missing values with the string "None"
for col in cat_cols:
    if col in df.columns:
        df[col] = df[col].fillna("None")

# Create dummy variables
df = pd.get_dummies(df, columns=[col for col in cat_cols if col in df.columns], drop_first=True)

# Remove outliers in PRICE (e.g., using IQR)
if 'PRICE' in df.columns:
    Q1 = df['PRICE'].quantile(0.25)
    Q3 = df['PRICE'].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    df = df[(df['PRICE'] >= lower) & (df['PRICE'] <= upper)]

# Join with gdf on parcel number (replace 'PARCELID' with your actual join key)
if 'PARCEL_NUMBER' in df.columns and 'PARCEL_NUMBER' in gdf.columns:
    merged = df.merge(gdf, on='PARCEL_NUMBER', how='inner')
else:
    print("PARCEL_NUMBER column not found in both dataframes for join.")

# Join distance featrues to dataframe

In [ ]:
sf = sf[sf['QU_FLG'] == 'Q']
sf = sf[sf['PRICE'] > 100]

sf['FRONTAGE'] = sf['FRONTAGE'].fillna('No Frontage')
sf['VIEWS'] = sf['VIEWS'].fillna('No View')

sf['SEAWALL'] = sf['SEAWALL'].map({'Yes': 1, 'No': 0})
sf['DLHL_YN'] = sf['DLHL_YN'].map({'Y': 1, 'N': 0})
sf['POOL'] = sf['POOL'].map({'Y': 1, 'N': 0})
sf['WATERFRONT_YN'] = sf['WATERFRONT_YN'].map({'Y': 1, 'N': 0})
sf['CLUBHOUSE'] = sf['CLUBHOUSE'].map({'Yes': 1, 'No': 0})
sf['MARINA'] = sf['MARINA'].map({'Yes': 1, 'No': 0})
sf['TENNIS'] = sf['TENNIS'].map({'Yes': 1, 'No': 0})

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from statsmodels.stats.outliers_influence import variance_inflation_factor

# 1. Basic info and summary
print("DataFrame shape:", df.shape)
display(df.head())
display(df.describe(include='all'))

# 2. Distribution of target variable
plt.figure(figsize=(8,4))
sns.histplot(df['PRICE'], kde=True)
plt.title('Distribution of PRICE')
plt.show()

# 3. Correlation matrix (all numeric columns)
corr = df.corr(numeric_only=True)
plt.figure(figsize=(12,8))
sns.heatmap(corr, annot=True, fmt=".2f", cmap='coolwarm')
plt.title('Correlation Matrix')
plt.show()

# 4. Correlation of each variable with target
target_corr = corr['PRICE'].sort_values(ascending=False)
print("Correlation of each variable with PRICE:")
display(target_corr)

# 5. Pairplot for top correlated features
top_corr_features = target_corr[1:6].index  # top 5 features (excluding PRICE itself)
sns.pairplot(df, vars=top_corr_features.insert(0, 'PRICE'))
plt.show()

# 6. Boxplots for categorical variables (if not too many)
cat_vars = [col for col in df.columns if df[col].dtype == 'object']
for col in cat_vars[:5]:  # show for first 5 categorical variables
    plt.figure(figsize=(8,4))
    sns.boxplot(x=col, y='PRICE', data=df)
    plt.title(f'PRICE by {col}')
    plt.xticks(rotation=45)
    plt.show()

# 7. Variance Inflation Factor (VIF) for multicollinearity
# Only use numeric columns and drop rows with NA
X = df.select_dtypes(include=[np.number]).dropna()
vif_data = pd.DataFrame()
vif_data["feature"] = X.columns
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
display(vif_data.sort_values("VIF", ascending=False))

# 8. Missing values heatmap
plt.figure(figsize=(12,6))
sns.heatmap(df.isnull(), cbar=False, yticklabels=False)
plt.title('Missing Values Heatmap')
plt.show()

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

# 1. Select top 5 features most correlated with PRICE
top_features = target_corr.index[1:6].tolist()  # Exclude 'PRICE' itself

# 2. Prepare data for regression
X = df[top_features]
y = df['PRICE']

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 3. Fit linear regression
lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)

# 4. Evaluate
print("Linear Regression Results:")
print("R^2:", r2_score(y_test, y_pred))
print("RMSE:", np.sqrt(mean_squared_error(y_test, y_pred)))
print("Coefficients:", dict(zip(top_features, lr.coef_)))

# 5. Geographically Weighted Regression (GWR)
# Requires mgwr: pip install mgwr
from mgwr.gwr import GWR, Sel_BW

# You need coordinates for GWR. If you have geometry, extract centroid x/y.
if 'geometry' in gdf.columns:
    gdf['x'] = gdf.geometry.centroid.x
    gdf['y'] = gdf.geometry.centroid.y
    # Merge coordinates into df
    df_coords = gdf[['PARCEL_NUMBER', 'x', 'y']]
    df = df.merge(df_coords, on='PARCEL_NUMBER', how='left')
elif 'x' in df.columns and 'y' in df.columns:
    pass  # Already present
else:
    raise ValueError("No coordinates found for GWR.")

coords = df[['x', 'y']].values

# Prepare X and y for GWR (must be numpy arrays, no NA)
X_gwr = df[top_features].values
y_gwr = df['PRICE'].values.reshape(-1, 1)

# Select bandwidth
bw = Sel_BW(coords, y_gwr, X_gwr).search()

# Fit GWR
gwr_model = GWR(coords, y_gwr, X_gwr, bw)
gwr_results = gwr_model.fit()

print("GWR Results:")
print("GWR R^2:", gwr_results.R2)
print("GWR Adj. R^2:", gwr_results.adj_R2)
print("GWR Coefficients (first 5 rows):")
print(gwr_results.params[:5])

In [ ]:
from mgwr.mgwr import MGWR
from mgwr.sel_bw import Sel_BW as Sel_BW_MGWR

# Prepare data (no missing values)
mask = ~np.isnan(X_gwr).any(axis=1) & ~np.isnan(y_gwr).flatten() & ~np.isnan(coords).any(axis=1)
X_mgwr = X_gwr[mask]
y_mgwr = y_gwr[mask]
coords_mgwr = coords[mask]

# Select bandwidths for MGWR
mgwr_bw = Sel_BW_MGWR(coords_mgwr, y_mgwr, X_mgwr, multi=True).search()

# Fit MGWR
mgwr_model = MGWR(coords_mgwr, y_mgwr, X_mgwr, mgwr_bw)
mgwr_results = mgwr_model.fit()

print("MGWR Results:")
print("MGWR R^2:", mgwr_results.R2)
print("MGWR Adj. R^2:", mgwr_results.adj_R2)
print("MGWR Bandwidths:", mgwr_bw)
print("MGWR Coefficients (first 5 rows):")
print(mgwr_results.params[:5])